Лабораторная работа 1. Группа № 4
Современные подходы к обработке и хранению больших данных. Формы представления данных, типы и виды данных.


1. CSV: Подсчёт числа выздоровевших в каждой стране за февраль. Вывод в формате «страна-число».

In [29]:
import pandas as pd
import sqlite3

covid_data = pd.read_csv("covid_19_data.csv")
covid_data_feb = covid_data.loc[covid_data.ObservationDate=='02/29/2020', ['Country/Region', 'Recovered']]
covid_data_jan = covid_data.loc[covid_data.ObservationDate=='01/31/2020', ['Country/Region', 'Recovered']]

res = covid_data_feb.groupby('Country/Region').Recovered.sum() - covid_data_jan.groupby('Country/Region').Recovered.sum()
res.fillna(0) #res.dropna()

Country/Region
Afghanistan              0.0
Algeria                  0.0
Australia                9.0
Austria                  0.0
Bahrain                  0.0
                        ... 
Thailand                23.0
UK                       8.0
US                       7.0
United Arab Emirates     5.0
Vietnam                 16.0
Name: Recovered, Length: 63, dtype: float64

С сортировкой по количеству:

In [30]:
sorted_res = res.sort_values(ascending=False)
print(sorted_res.dropna())

Country/Region
Mainland China          39065.0
Singapore                  72.0
Italy                      46.0
Hong Kong                  33.0
Japan                      31.0
South Korea                27.0
Thailand                   23.0
Malaysia                   18.0
Vietnam                    16.0
Germany                    16.0
France                     12.0
Taiwan                      9.0
Australia                   9.0
Macau                       8.0
UK                          8.0
US                          7.0
Canada                      6.0
United Arab Emirates        5.0
India                       3.0
Russia                      2.0
Cambodia                    1.0
Philippines                 1.0
Sri Lanka                   1.0
Finland                     1.0
Nepal                       1.0
Sweden                      0.0
Name: Recovered, dtype: float64


Наибольшое количество выздоровевших приходится на Китай, что очевидно, так как пандемия началась там.

2. SQLite: Показать уровень продаж за ноябрь, число проданных билетов и среднюю цену за билет

In [31]:
import sqlite3
import pandas as pd

con = sqlite3.connect("movies.sqlite")
cur = con.cursor()

df = pd.read_sql('''
                       SELECT case strftime('%m', date) when '11' then 'November' else '' end m, 
                           strftime('%Y', date) y, 
                           sum(revenue) revenue, 
                           sum(tickets) as count, 
                           sum(revenue)/sum(tickets) avg_price
                       FROM weekly_box_office
                       WHERE strftime("%m", date) = "11"
                       GROUP BY m, y;
                        ''', con)

df

,m,y,revenue,count,avg_price
0,November,2017,616841237,2672924,230
1,November,2018,671393703,2850397,235
2,November,2019,646863217,2756609,234


или, без использования агрегатных функций SQL:

In [32]:
con = sqlite3.connect("movies.sqlite")
movies = pd.read_sql_query("SELECT * from weekly_box_office", con)
movies['date'] = pd.to_datetime(movies['date'])
movies_nov = movies.loc[movies['date'].dt.month == 11, ['date', 'revenue', 'tickets']]
revenue = movies_nov.groupby(movies_nov.date.dt.year)['revenue'].sum()
tickets = movies_nov.groupby(movies_nov.date.dt.year)['tickets'].sum()
revenue
for i in range (2017, 2020):
    print(i, 'revenue: ', revenue[i], ' tickets: ', tickets[i],  ' avg: ', revenue[i]/tickets[i])

2017 revenue:  616841237  tickets:  2672924  avg:  230.77395279476707
2018 revenue:  671393703  tickets:  2850397  avg:  235.54392703893527
2019 revenue:  646863217  tickets:  2756609  avg:  234.65903833296633


Из данной статистики следует, что наибольшее прибыльным годом был 2018 (большая выручка, количество проданных билетов, наибольшая цена билета), следом по прибыльности идет 2019 год, менее прибыльным, из приведенных годов, является 2017 год, уступая 2018 году на 177 тыс. проданных билетов.

3. JSON: Показать уровень фертильности стран, названия которых начинаются на “I”

In [33]:
import pandas as pd

df = pd.read_json('unece.json')
filtered_df = df[(df['Country'].str.startswith('I'))].copy()
filtered_df['Prev Total fertility rate'] = filtered_df['Total fertility rate'].shift(1)

for country in filtered_df['Country'].unique():
    print("Год \t Страна \t Фертильность \t Отклонение")
    for index, row in filtered_df[filtered_df['Country'] == country].iterrows():
        diff = row['Total fertility rate'] - row['Prev Total fertility rate']

        if not pd.isna(row['Total fertility rate']):
            if pd.isna(diff):
                print(f'{row["Year"]:4}', "\t", f'{row["Country"]:10}', "\t", f"{row['Total fertility rate']:.2f}")
            else:
                print(f'{row["Year"]:4}', "\t", f'{row["Country"]:10}', "\t", f"{row['Total fertility rate']:.2f}", "\t\t", "(", f"{diff:.2f}", ")")
    print("")


Год 	 Страна 	 Фертильность 	 Отклонение
2000 	 Iceland    	 2.08
2001 	 Iceland    	 1.95 		 ( -0.13 )
2002 	 Iceland    	 1.93 		 ( -0.02 )
2003 	 Iceland    	 1.99 		 ( 0.06 )
2004 	 Iceland    	 2.04 		 ( 0.05 )
2005 	 Iceland    	 2.05 		 ( 0.01 )
2006 	 Iceland    	 2.08 		 ( 0.03 )
2007 	 Iceland    	 2.09 		 ( 0.01 )
2008 	 Iceland    	 2.15 		 ( 0.06 )
2009 	 Iceland    	 2.23 		 ( 0.08 )
2010 	 Iceland    	 2.20 		 ( -0.03 )
2011 	 Iceland    	 2.02 		 ( -0.18 )
2012 	 Iceland    	 2.04 		 ( 0.02 )
2013 	 Iceland    	 1.93 		 ( -0.11 )
2014 	 Iceland    	 1.93 		 ( 0.00 )

Год 	 Страна 	 Фертильность 	 Отклонение
2000 	 Ireland    	 1.89
2001 	 Ireland    	 1.94 		 ( 0.05 )
2002 	 Ireland    	 1.97 		 ( 0.03 )
2003 	 Ireland    	 1.96 		 ( -0.01 )
2004 	 Ireland    	 1.93 		 ( -0.03 )
2005 	 Ireland    	 1.86 		 ( -0.07 )
2006 	 Ireland    	 1.91 		 ( 0.05 )
2007 	 Ireland    	 2.01 		 ( 0.10 )
2008 	 Ireland    	 2.06 		 ( 0.05 )
2009 	 Ireland    	 2.06 		 ( 0.00 )
2010 	 I

В наблюденной статистике наблюдается:
1. Iceland. Резкое падение фертильности в 2001, 2011 и 2013 годах. Рост фертильности в 2003, 2004, 2008 и 2009 годах. Рост фертильности идет по 2 года. Если в 2009 году фертильность была максимальной, то после него идет стабильное падение фертильности (в 2014 году фертильность меньше, чем в 2000).
2. Ireland. Схожая ситуация с Iceland, также максимальная фертильность в 2009 года с последующем падением фертильности. Упад фертильности не такой резкий, как в Iceland.
3. Israel. Наибольший рост уровня фертильности наблюдается в 2010 году (с последующем падением в 2011 г.). На временном интервале отмечается рост уровня фертильности с компенсацией после тех годов, где рост уровня фертильности был большим. В 2014 году наблюдается наибольший уровень фертильности.
4. Italy. Отмечается стабильный рост уровня фертильности с 2002 года по 2010 год, после чего следует стабильный спад уровня фертильности (2011-2014 г.). Несмотря на это, в 2014 году фертильность превышает уровень фертильности в 2006 году и ранее.